In [ ]:
# Usually not recommended, but they simply add unneeded verbosity to the output
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler 
from sklearn.linear_model import *
from sklearn.svm import *
import sympy as sp

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr, spearmanr

In [ ]:
# Offsets are expressed in days
offset_1 =1
offset_2 = 24
model_type = 'elasticnet_AR_scaled'
if offset_2:
    model = f'{model_type}_{offset_1}d_{offset_2}d'
else:
    model = f'{model_type}_{offset_1}d'

In [ ]:
correspondences = {
    'V': 'V_t',
    'N': '\\rho_t',
    'T': 'T_t',
    'BT': 'B_T',
    'BR': 'B_R',
    'BN': 'B_N', 
}

def parse_name(n):
    new = n.split('-')
    return new[0]+f'_t-{new[1]}'

In [ ]:
def tabulate_series(df, freq='1h', inputs=1, offset=1, second_offset = None):
    '''
        Tabulate time series data contained in a matrix to generate vectors usable by
        the algorithms included in this work.
    '''
    src = df.copy().shift(offset+inputs)
    new_c = src.columns.values.copy()
    for i in range(len(new_c)):
        new_c[i] = new_c[i]+f'-{offset+inputs}'
    src.columns = new_c
    for i in tqdm(range(offset+inputs-1, offset-1, -1), desc='Data tabulation (1st pass)'):
        new_df = df.shift(i, freq=freq)
        new_c = new_df.columns.values.copy()
        new_df.columns = new_c
        for j in range(len(new_c)):
            new_c[j] = new_c[j]+f'-{i}'
        src = src.join(new_df)
    if second_offset:
        for i in tqdm(range(second_offset+inputs-1, second_offset-1, -1), desc='Data tabulation (2nd pass)'):
            new_df = df.shift(i, freq=freq)
            new_c = new_df.columns.values.copy()
            new_df.columns = new_c
            for j in range(len(new_c)):
                new_c[j] = new_c[j]+f'-{i}'
            src = src.join(new_df)
    return src.join(df).dropna(how='any')

# Support functions to enable generation of the correct J and K values.
# Do not modify.
def compute_num_inputs(days=1, freq='1d'):
    if 'd' in freq:
        divisor = float(freq.replace('d', ''))
    if 'h' in freq:
        divisor = float(freq.replace('h', ''))/24
    if 'min' in freq:
        divisor = float(freq.replace('min', ''))/(24*60)
    if 's' in freq:
        divisor = float(freq.replace('s', ''))/(24*60*60)
    return round(days/divisor)

def compute_window_size(freq='1d'):
    if 'd' in freq:
        return float(freq.replace('d', ''))*24
    else:
        return float(freq.replace('h', ''))

# Main algorithm
def explore_aggregation_space(aggregations, min_input_days=1, max_input_days=5, 
                              min_offset_days=1, max_offset_days=4, special_offset = None):
    '''
        Algorithmically generate as many configurations as needed for the selected model.
        To change the model use, simply comment the one currently enabled an uncomment the
        selected sklearn model.
    '''
    error_metrics = {}
    expressions = {'l1_ratio': []}
    expressions['latex'] = []
    for agg in tqdm(aggregations, desc='Aggregation sizes explored'):
        # prepare dictionary for error metric indexing using pandas later
        error_metrics[f'{agg}_rmse'] = {}
        error_metrics[f'{agg}_mae'] = {}
        error_metrics[f'{agg}_corr'] = {}
        error_metrics[f'{agg}_r2'] = {}
        
        # generate a copy of the original dataset exclusive to the loop iteration
        surrogate_data = wind_data.copy()
        surrogate_data_non_smoothed = surrogate_data.copy() # set aside an unsmoothed copy of data
        # aggregate by time
        surrogate_data = surrogate_data.rolling(round(compute_window_size(agg)), center=True).mean().dropna(how='any')
        
        # now we loop over the different configurations for the model
        for inpt in tqdm(range(min_input_days, max_input_days+1), 
                           desc=f'Number of days in input for {agg} aggregation explored'):
            input_size = compute_num_inputs(inpt, agg)
            for offset in tqdm(range(min_offset_days, max_offset_days+1), desc='Offsets explored'):
                offset_size = offset*24

                # tabulate data for usage according to our configuration
                if special_offset:
                    tabulated = tabulate_series(surrogate_data, agg, input_size, offset_size, special_offset*24)
                    tabulated_non_smoothed = tabulated#tabulate_series(surrogate_data_non_smoothed, agg, input_size, offset_size,
#                                                             special_offset*24)
                else:
                    tabulated = tabulate_series(surrogate_data, agg, input_size, offset_size)
                    tabulated_non_smoothed = tabulated#tabulate_series(surrogate_data_non_smoothed, agg, input_size, offset_size)
                    
                
                # exclude specific columns from potential inputs
                exclude = ['lower_loc', 'upper_loc', 'height', 'mag_skewness', 'mag_flux', 'BR', 'BT', 'BN', 'unipolarity',
                          'N', 'T', 'log_{10}T']
                inputs = tabulated.columns.values.tolist()
                # very dirty exception handling, but we know there will only be
                # exceptions here if the column doesn't exist, so we can just
                # skip any that are like that and keep going until we're done with the
                # entire list
                for o in tqdm(targets, desc='Filtering targets out from input dataset'):
                    try:
                        inputs.remove(o)
                    except:
                        continue
                for o in tqdm(exclude, desc='Removing banned entries from input dataset'):
                    try:
                        inputs.remove(o)
                    except:
                        continue
                
                # split tabulated data between inputs and targets
                input_df = tabulated[inputs]
                output_df = tabulated[targets]
                
                test_data_in = []
                test_data_out = []
                for row in tqdm(val_info.values, desc='Generating dataset split'):
                    start = row[0]
                    end = row[1]
                    new_df_in = input_df[input_df.index >= start].copy()
                    new_df_in = new_df_in[new_df_in.index <= end].copy()
                    new_df_out = tabulated_non_smoothed[targets][tabulated_non_smoothed.index >= start].copy()
                    new_df_out = new_df_out[new_df_out.index <= end].copy()
                    test_data_in.append(new_df_in)
                    test_data_out.append(new_df_out)
                
                inpt_scaler = MinMaxScaler()
                outpt_scaler = MinMaxScaler()
                    
                X_test = pd.concat(test_data_in)
                y_test = pd.concat(test_data_out)
                X_train = input_df[~input_df.index.isin(X_test.index)].copy()
                y_train = output_df[~output_df.index.isin(X_test.index)].copy()
                
                
                inpt_scaler.fit(X_train)
                outpt_scaler.fit(y_train)
                
                X_train = pd.DataFrame(inpt_scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
                y_train = pd.DataFrame(outpt_scaler.transform(y_train), index=y_train.index, columns=y_train.columns)
                X_test = pd.DataFrame(inpt_scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
                
                model = ElasticNetCV(l1_ratio=[i for i in np.arange(0.1, 1.0, .03)], 
                                     verbose=0, 
                                     n_jobs=2)
#                 model = SVR(kernel='rbf')
                model.fit(X_train, y_train)
                
#                 try:
#                     print(f'Support vectors: {model.support_vectors_}')
#                 except:
#                     pass
    
                try:
                    expressions['l1_ratio'].append(model.l1_ratio_)
                except Exception as e:
                    print(f'Exception when extracting alpha parameter for {inpt} inputs, {offset} outputs case:\n\t{e}')
                    expressions['l1_ratio'].append(np.nan)
                pred = outpt_scaler.inverse_transform(model.predict(X_test).reshape(-1, 1))
                #pred = model.predict(X_test).reshape(-1, 1)
                pred = 10**pred
                y_test = 10**y_test.values[:, 0]
                r2 = r2_score(y_test, pred)
                rmse = mean_squared_error(y_test, pred, squared=False)
                mae = mean_absolute_error(y_test, pred)
                pearson, p = pearsonr(y_test, pred)
                
                del X_train
                del y_train
                del X_test
                del y_test
                del tabulated
                if not tabulated_non_smoothed is None:
                    del tabulated_non_smoothed
                
                error_metrics[f'{agg}_rmse'][f'{(inpt, offset)}'] = rmse
                error_metrics[f'{agg}_mae'][f'{(inpt, offset)}'] = mae
                error_metrics[f'{agg}_corr'][f'{(inpt, offset)}'] = pearson
                error_metrics[f'{agg}_r2'][f'{(inpt, offset)}'] = r2
               
                try:
                    exprs = {}
                    for t in range(len(targets)):
                        symb_expr = ''
                        for s in model.feature_names_in_:
                            try:
                                symb_expr += correspondences[s]+' '
                            except:
                                symb_expr += parse_name(s)+' '
                        symbols = sp.symbols(symb_expr)
                        coefs = []
                        mcoefs = model.coef_
                        for i in range(len(mcoefs)):
                            coefs.append(mcoefs[i]*symbols[i])
                        expr = coefs[0]
                        for c in coefs[1:]:
                            expr += c
                        expr += model.intercept_
                        exprs[targets[t]] = expr
                    for s, exp in exprs.items():
                        expressions['latex'].append(sp.latex(sp.Eq(sp.symbols(correspondences[s]), exp.simplify())))
                except Exception as e:
                    print(f'Exception when extracting equation parameter for {inpt} inputs, {offset} outputs case:\n\t{e}')
                    expressions['latex'].append('-')
    
    return error_metrics, expressions

In [ ]:
# Load CSV with validation info. Provided under the 'data' folder
val_info = pd.read_csv('data/SW_comparison_validation_info.csv', sep=';')[['Start date', 'End date']]
val_info = val_info[len(val_info)//2:]
val_info

In [ ]:
# Load dataset with measurements, and ensure no element is out of range.
# Data should be retrieved from OmniWeb and placed under the 'data' folder.
# The dataset used is provided by default.
wind_data = pd.read_csv('data/omni_1hr_clean.csv', index_col=0, parse_dates=[0])
wind_data['V'] = wind_data['V'].apply(np.log10)
wind_data['T'] = wind_data['T'].apply(np.log10)
wind_data = wind_data[wind_data.index >= '2011-01-01']
wind_data = wind_data[wind_data.index < '2018-01-01']
display(wind_data)
new_idx = pd.date_range(wind_data.index[0], wind_data.index[-1], freq='1h')
wind_data = wind_data.reindex(new_idx, fill_value=np.nan)
wind_data

In [ ]:
aggregations=['1h']
targets = ['V']

In [ ]:
if offset_2:
    error_metrics, expressions = explore_aggregation_space(aggregations, min_offset_days=offset_1, 
                                                           max_offset_days=offset_1+2, special_offset=offset_2,
                                                          max_input_days=5)
else:    
    error_metrics, expressions = explore_aggregation_space(aggregations, min_offset_days=offset_1, 
                                                           max_offset_days=offset_1+2,
                                                          max_input_days=5)

In [ ]:
em_df = pd.DataFrame(error_metrics)
em_df['1h_corr'] = em_df['1h_corr'].apply(lambda x: x[0])
em_df

In [ ]:
metrics = ['rmse', 'mae', 'corr', 'r2']
metric_translations = {
    'rmse': 'RMSE (km/s)',
    'mae': 'MAE (km/s)',
    'corr': 'Pearson\'s correlation coefficient',
    'r2': 'R$^{2}$'
}

aggregation_translation = {
    '1d': '1 day',
    '22h': '22 hours',
    '20h': '20 hours',
    '18h': '18 hours',
    '16h': '16 hours',
    '14h': '14 hours',
    '12h': '12 hours',
    '10h': '10 hours',
    '8h': '8 hours',
    '6h': '6 hours',
    '4h': '4 hours',
    '3h': '3 hours',
    '2h': '2 hours',
    '1h': '1 hour'
}

for m in metrics:
    try:
        select = []
        for a in aggregations:
            select.append(f'{a}_{m}')
        plt.figure(figsize=(15, 5.5))
        xlabels = []
        for c in em_df[select].columns:
            xlabels.append(aggregation_translation[c.split('_')[0]])
        sns.heatmap(em_df[select], annot=True, xticklabels=xlabels)
        plt.title(f'Comparison of {metric_translations[m]} per configuration')
        plt.xlabel('Aggregation size')
        plt.ylabel('Input and offset length (days)')
        plt.show()
        plt.close()
    except:
        continue

In [ ]:
idx = []
for agg in aggregations:
    for v in em_df.index.values:
        idx.append(f'{agg}_{v}')
expr_df = pd.DataFrame(expressions, index=idx)
expr_df

In [ ]:
expr_df.groupby('l1_ratio').count()

In [ ]:
sns.histplot(expr_df['l1_ratio'], kde=True)
plt.suptitle('Number of expressions per L1 ratio')
plt.title(f'(N = {len(expr_df)})')
plt.xlabel('L1 ratio')
plt.show()
plt.close()
sns.histplot(expr_df['l1_ratio'], kde=True, stat='probability')
plt.suptitle('Probability of expressions per L1 ratio')
plt.title(f'(N = {len(expr_df)})')
plt.xlabel('L1 ratio')
plt.show()
plt.close()

In [ ]:
em_df.to_csv(f'results/performance_metrics_{model}.csv')
expr_df.to_csv(f'results/expressions_{model}.csv')